In [1]:
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


Add Adam Yala's code for model creation to Python path

In [2]:
import sys
sys.path.append('../OncoNet_Public')
sys.path.append('..')

In [3]:
import torch
print(torch.cuda.is_available())

True


Load MIT model and populate it with pretrained weights.

In [4]:
import onconet.utils.parsing_with_line as parsing
import onconet.models.factory as model_factory
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

def init_model(multi_gpu=False):
    # A long string to setup the Resnet model.
    cmd_line = '''
    --model_name custom_resnet --pool_name GlobalMaxPool 
    --block_layout BasicBlock,2 BasicBlock,2 BasicBlock,2 BasicBlock,2 
    --dropout 0 --num_chan 3 --pretrained_on_imagenet
    '''
    args = parsing.parse_args(cmd_line)
    model = model_factory.get_model(args)
    model = model._model
    model.load_state_dict(
        torch.load(
            'mods/mgh_mammo_cancer_5yr_risk_img_only_aug08_2018_state.pt'
        ))
    if multi_gpu:
        model = torch.nn.DataParallel(model)
    model = model.to(device)
    return model

cuda


In [5]:
model = init_model(multi_gpu=False)

Loaded pretrained_weights for 100 out of 122 parameters.


Test it with a dummy image.

In [6]:
dummy_img = torch.zeros(1, 3, 2048, 1664).to(device)
dummy_img.shape

torch.Size([1, 3, 2048, 1664])

In [7]:
logit_, hidden_, x_ = model(dummy_img)

In [8]:
logit_

tensor([[-1.5265, -1.0262]], device='cuda:0', grad_fn=<AddmmBackward>)

convert logits to probabilities

In [9]:
import torch.nn.functional as F
F.softmax(logit_)

tensor([[0.3775, 0.6225]], device='cuda:0', grad_fn=<SoftmaxBackward>)

### Finetune MIT model on MSSM data

These are my custom functions to convert images to PyTorch tensors

In [10]:
from mit_risk_data import MammoBCRiskDataset, Resize, FaceLeft, NormalizePix, ToTensor3D, mammo_collate

Load the 3+ yr dataset. There are three datasets in total: 1 yr, 1+ yr and 3+ yr.

In [11]:
risk_dataset_t3 = torch.load('../time_set/risk_pred_GEHolo_4view_matchedSepCase_T3.pt')

In [12]:
len(risk_dataset_t3)

256

In [13]:
sample = risk_dataset_t3[8]

In [14]:
sample.keys()

dict_keys(['subject', 'exam', 'label', 'machine', 'age', 'race', 'bmi', 'birads', 'libra', 'years_before_dx', 'images'])

Get labels for all samples for stratified splitting. This can take a while, so I save the labels to load them later.

In [15]:
# ys_t3 = np.array([ sample['label'].item() for sample in risk_dataset_t3])

In [16]:
# np.save('tmp/ys_t3.npy', ys_t3)

In [17]:
# Some helper functions
import matplotlib.pyplot as plt
from IPython.display import Image
import pickle
import os

# Store variables with pickle
def load_pkl(filename):
    var = pickle.load(open(filename, 'rb'))
    print('Loaded data from:', filename)
    return var

def store_pkl(var, filename):
    pickle.dump(var, open(filename, 'wb'), protocol=4)
    print('Stored data in:', filename)

# Display images
def display_image(num):
    return Image(filename='images_png_2048x1664/{0:08d}.png'.format(num)) 

def display_tensor(tensor):
    plt.imshow(tensor.permute(1, 2, 0))

In [18]:
if not os.path.exists('ys_t3.pkl'):
    ys_t3 = np.array([ sample['label'].item() for sample in risk_dataset_t3])
    store_pkl(ys_t3, 'ys_t3.pkl')
else:
    ys_t3 = load_pkl('ys_t3.pkl')

Stored data in: ys_t3.pkl


\# of controls and cases, the size of control is exactly three times the size of case.

In [19]:
np.unique(ys_t3, return_counts=True)

(array([0, 1]), array([192,  64]))

Test train-val-test split for K-fold cross-validation

In [20]:
from sklearn.model_selection import StratifiedKFold, train_test_split

In [21]:
n_folds = 3
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=12345)
for train_ix_, test_ix in skf.split(np.ones((len(ys_t3), 1)), ys_t3):
    train_y = ys_t3[train_ix_]
    test_prop = (1/n_folds)/(1 - 1/n_folds)
    train_ix, val_ix = train_test_split(train_ix_, test_size=test_prop, 
                                        stratify=train_y, 
                                        random_state=12345)
    print('train - {}, val - {}, test - {}'.format(
        np.bincount(ys_t3[train_ix]), np.bincount(ys_t3[val_ix]), np.bincount(ys_t3[test_ix])))

train - [64 21], val - [64 21], test - [64 22]
train - [64 21], val - [64 22], test - [64 21]
train - [64 21], val - [64 22], test - [64 21]


In [22]:
from torch.utils.data import Subset
train_dataset = Subset(risk_dataset_t3, train_ix)
val_dataset = Subset(risk_dataset_t3, val_ix)
test_dataset = Subset(risk_dataset_t3, test_ix)

In [23]:
train_y = ys_t3[train_ix]
val_y = ys_t3[val_ix]
test_y = ys_t3[test_ix]

Use weighted sampler for the train dataloader

In [24]:
f0, f1 = np.bincount(train_y)
train_w = np.zeros_like(train_y, dtype='float')
train_w[train_y==0] = 1/f0
train_w[train_y==1] = 1/f1

In [25]:
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader

cpu_threads = 4
batch_size = 1

weighted_sampler = WeightedRandomSampler(
    train_w, len(train_y)//batch_size*batch_size, 
    replacement=True)
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, 
    num_workers=cpu_threads, sampler=weighted_sampler,
    collate_fn=mammo_collate)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, 
    num_workers=cpu_threads, drop_last=False,
    collate_fn=mammo_collate)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, 
    num_workers=cpu_threads, drop_last=False,
    collate_fn=mammo_collate)

Test loading a batch of samples

In [26]:
bat_sample = next(iter(train_loader))

In [27]:
bat_sample.keys()

dict_keys(['subject', 'exam', 'machine', 'age', 'race', 'bmi', 'birads', 'libra', 'years_before_dx', 'label', 'images'])

In [28]:
bat_sample['images']['same-cc'].shape, bat_sample['images']['same-mlo'].shape

(torch.Size([1, 1, 2048, 1664]), torch.Size([1, 1, 2048, 1664]))

In [29]:
model.eval()
with torch.no_grad():
    bat_logit, _, _ = model(bat_sample['images']['same-cc'].float().to(device))

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 2048, 1664] to have 3 channels, but got 1 channels instead

In [ ]:
F.softmax(bat_logit)[:, 1]

Define a function to calculate avg loss and AUC for one sweep of a dataloader

In [ ]:
import torch.nn as nn
from sklearn.metrics import roc_auc_score

def val_loss(model, val_loader, return_auc=False):
    '''Avg loss and AUC for an entire data loader
    '''
    criterion_ = nn.NLLLoss(reduction='sum').to(device)
    model.eval()
    val_loss, n = 0, 0
    y_pool, p_pool = [], []
    with torch.no_grad():
        for batch in val_loader:
            bat_X_1 = batch['images']['same-cc'].float().to(device)
            bat_X_2 = batch['images']['same-mlo'].float().to(device)
            bat_y = batch['label'].long().to(device)
            bat_logit_1, _, _ = model(bat_X_1)
            bat_logit_2, _, _ = model(bat_X_2)
            bat_logp_1 = F.log_softmax(bat_logit_1)
            bat_logp_2 = F.log_softmax(bat_logit_2)
            loss_1 = criterion_(bat_logp_1, bat_y)
            loss_2 = criterion_(bat_logp_2, bat_y)
            val_loss += loss_1.item()
            val_loss += loss_2.item()
            n += len(bat_X_1)
            n += len(bat_X_2)
            y_pool.append(bat_y)
            y_pool.append(bat_y)  # same label, different view.
            p_pool.append(torch.exp(bat_logp_1))
            p_pool.append(torch.exp(bat_logp_2))
        if return_auc:
            ys = torch.cat(y_pool)
            ps = torch.cat(p_pool)
            ys = ys.cpu().detach().numpy()
            ps = ps.cpu().detach().numpy()
            val_auc = roc_auc_score(ys, ps[:, 1])
            return val_loss/n, val_auc
    return val_loss/n

In [ ]:
from time import time

start = time()
train_loss_, train_auc_ = val_loss(model, train_loader, return_auc=True)
print('Init avg train loss={:.3f}, auc={:.3f}'.format(train_loss_, train_auc_))
val_loss_, val_auc_ = val_loss(model, val_loader, return_auc=True)
print('Init avg val loss={:.3f}, auc={:.3f}'.format(val_loss_, val_auc_))
test_loss_, test_auc_ = val_loss(model, test_loader, return_auc=True)
print('Init avg test loss={:.3f}, auc={:.3f}'.format(test_loss_, test_auc_))
duration = time() - start
print('Time elapsed={:.1f}'.format(duration))

In [ ]:
from torch.optim import Adam, SGD
criterion = nn.NLLLoss(reduction='mean').to(device)
optimizer = Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)

**Optimization Notes:**
- Adam with lr=1e-5 worked fine. Train and val loss showed robust decrease. Val AUC somehow decreased in the beginning but clearly trended upward. Best val AUC=0.55; test AUC increased to 0.59.
- Adam with lr=1e-4 clearly sped up learning with training loss quickly driven towards zero. This led to overfitting but the val AUCs all looked good with the best val AUC=0.61. The test AUC was 0.62 although the test loss was high.
- 2nd try with lr=1e-4, best model saved, led to val AUC=0.63 but test AUC=0.52.
- 2nd try with lr=1e-5, best model saved, led to val AUC=0.56; test AUC=0.57.
- Adam with lr=1e-6 led to slow decrease of train and val loss; val AUC hovers around 0.5. Test AUC was 0.59.
- Increased lr to 3e-6, got similar results.

Based on 10 epochs.

**Finetuning test run**

In [ ]:
start_epoch = 0
epochs = 20
train_loss = 0
check_iters = 10
best_name = 'best_model.pt'
best_auc = .0
start = time()
for i in range(start_epoch, start_epoch + epochs):
    for j, batch in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        # Each iteration is equivalent to 2*batch_size.
        bat_X_1 = batch['images']['same-cc'].float().to(device)
        bat_X_2 = batch['images']['same-mlo'].float().to(device)
        bat_y = batch['label'].long().to(device)
        # forward-backward CC view images.
        bat_logit_1, _, _ = model(bat_X_1)
        bat_logp_1 = F.log_softmax(bat_logit_1)
        loss = criterion(bat_logp_1, bat_y)
        loss.backward()
        train_loss += loss.item()/2
        # forward-backward MLO view images.
        bat_logit_2, _, _ = model(bat_X_2)
        bat_logp_2 = F.log_softmax(bat_logit_2)
        loss = criterion(bat_logp_2, bat_y)
        loss.backward()
        train_loss += loss.item()/2
        # accumulate gradients from both CC and MLO images and 
        # then take a step to update.
        optimizer.step()
        total_iters = i*len(train_loader) + j + 1
        if total_iters%check_iters == 0:
            avg_val_loss, val_auc = val_loss(model, val_loader, return_auc=True)
            avg_train_loss = train_loss/check_iters
            print("Iter={}, avg train loss={:.3f}, avg val loss={:.3f}, auc={:.3f}".format(
                total_iters, avg_train_loss, avg_val_loss, val_auc))
            if val_auc > best_auc:
                best_auc = val_auc
                torch.save(model.state_dict(), best_name)
                print("Best model saved.")
            train_loss = 0
duration = time() - start
print('Time elapsed={:.1f}'.format(duration))
model.load_state_dict(torch.load(best_name))
print('Best model loaded.')

In [ ]:
avg_test_loss, test_auc = val_loss(model, test_loader, return_auc=True)
print('Finetuned avg test loss={:.3f}, auc={:.3f}'.format(avg_test_loss, test_auc))

Define a function to calculate AUC based on the max score of the four views

In [ ]:
import torch.nn as nn
from sklearn.metrics import roc_auc_score

def test_max_auc(model, test_loader):
    '''AUC based on max prediction of 4 views
    '''
    model.eval()
    y_pool, p_pool = [], []
    with torch.no_grad():
        for batch in test_loader:
            bat_X_1 = batch['images']['same-cc'].float().to(device)
            bat_X_2 = batch['images']['same-mlo'].float().to(device)
            bat_X_3 = batch['images']['opposite-cc'].float().to(device)
            bat_X_4 = batch['images']['opposite-mlo'].float().to(device)
            bat_y = batch['label'].long().to(device)
            bat_logit_1, _, _ = model(bat_X_1)
            bat_logit_2, _, _ = model(bat_X_2)
            bat_logit_3, _, _ = model(bat_X_3)
            bat_logit_4, _, _ = model(bat_X_4)
            bat_p_1 = F.softmax(bat_logit_1)[:, 1]
            bat_p_2 = F.softmax(bat_logit_2)[:, 1]
            bat_p_3 = F.softmax(bat_logit_3)[:, 1]
            bat_p_4 = F.softmax(bat_logit_4)[:, 1]
            bat_p = torch.stack(
                [bat_p_1, bat_p_2, bat_p_3, bat_p_4], 
                dim=1)
            p_pool.append(bat_p)
            y_pool.append(bat_y)
        ys = torch.cat(y_pool)
        ps = torch.cat(p_pool)
        max_ps = ps.max(1).values
        ys = ys.cpu().detach().numpy()
        max_ps = max_ps.cpu().detach().numpy()
        test_auc = roc_auc_score(ys, max_ps)
        return test_auc

In [ ]:
test_auc_m = test_max_auc(model, test_loader)
print('Finetuned max-score-based auc={:.3f}'.format(test_auc_m))

### Wrap up train-test routines

In [ ]:
def train(model, train_loader, val_loader, best_name, 
          epochs=3, lr=1e-6, weight_decay=1e-4, 
          check_iters=30, log_name=None):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    writer = SummaryWriter(log_name) if log_name is not None else None
    train_loss = 0
    best_auc = .0
    for i in range(epochs):
        for j, batch in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            # Each iteration is equivalent to 2*batch_size.
            bat_X_1 = batch['images']['same-cc'].float().to(device)
            bat_X_2 = batch['images']['same-mlo'].float().to(device)
            bat_y = batch['label'].long().to(device)
            # forward-backward CC view images.
            bat_logit_1, _, _ = model(bat_X_1)
            bat_logp_1 = F.log_softmax(bat_logit_1)
            loss = criterion(bat_logp_1, bat_y)
            loss.backward()
            train_loss += loss.item()/2
            # forward-backward MLO view images.
            bat_logit_2, _, _ = model(bat_X_2)
            bat_logp_2 = F.log_softmax(bat_logit_2)
            loss = criterion(bat_logp_2, bat_y)
            loss.backward()
            train_loss += loss.item()/2
            optimizer.step()
            total_iters = i*len(train_loader) + j + 1
            if total_iters%check_iters == 0:
                avg_val_loss, val_auc = val_loss(model, val_loader, return_auc=True)
                avg_train_loss = train_loss/check_iters
                print("Iter={}, avg train loss={:.3f}, "
                      "avg val loss={:.3f}, auc={:.3f}".format(
                    total_iters, avg_train_loss, avg_val_loss, val_auc))
                if val_auc > best_auc:
                    best_auc = val_auc
                    torch.save(model.state_dict(), best_name)
                    print("Best model saved.")
                if writer is not None:
                    writer.add_scalar('Loss/train', avg_train_loss, total_iters)
                    writer.add_scalar('Loss/val', avg_val_loss, total_iters)
                train_loss = 0
#                 break
    print("Best model loaded.")
    model.load_state_dict(torch.load(best_name))

In [ ]:
def do_test(model, test_loader):
    '''Avg loss and AUC for an entire data loader
    '''
    model.eval()
    subject_list, exam_list = [], []
    pred_list, label_list, machine_list = [], [], []
    age_list, race_list, bmi_list = [], [], []
    birads_list, libra_list = [], []
    with torch.no_grad():
        for batch in test_loader:
            bat_X_1 = batch['images']['same-cc'].float().to(device)
            bat_X_2 = batch['images']['same-mlo'].float().to(device)
            bat_X_3 = batch['images']['opposite-cc'].float().to(device)
            bat_X_4 = batch['images']['opposite-mlo'].float().to(device)
            bat_y = batch['label'].long().to(device)
            bat_logit_1, _, _ = model(bat_X_1)
            bat_logit_2, _, _ = model(bat_X_2)
            bat_logit_3, _, _ = model(bat_X_3)
            bat_logit_4, _, _ = model(bat_X_4)
            bat_p_1 = F.softmax(bat_logit_1)[:, 1]
            bat_p_2 = F.softmax(bat_logit_2)[:, 1]
            bat_p_3 = F.softmax(bat_logit_3)[:, 1]
            bat_p_4 = F.softmax(bat_logit_4)[:, 1]
            bat_p = torch.stack(
                [bat_p_1, bat_p_2, bat_p_3, bat_p_4], 
                dim=1)
            subject_list.append(batch['subject'])
            exam_list.append(batch['exam'])
            pred_list.append(bat_p)
            label_list.append(batch['label'])            
            machine_list.append(batch['machine'])
            age_list.append(batch['age'])
            race_list.append(batch['race'])
            bmi_list.append(batch['bmi'])
            birads_list.append(batch['birads'])
            libra_list.append(batch['libra'])            
    return(subject_list, exam_list, pred_list, label_list, 
           machine_list, age_list, race_list, bmi_list, 
           birads_list, libra_list)

**Risk prediction 3+ years**

In [ ]:
from torch.optim import Adam

criterion = nn.NLLLoss(reduction='mean').to(device)
cpu_threads = 4
batch_size = 4
n_folds = 5
epochs = 20
subject_pool, exam_pool = [], []
pred_pool, label_pool, machine_pool = [], [], []
age_pool, race_pool, bmi_pool = [], [], []
birads_pool, libra_pool = [], []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=12345)
fold = 0
for train_ix_, test_ix in skf.split(np.ones((len(ys_t3), 1)), ys_t3):
    fold += 1
    # train-val-test idx.
    train_y = ys_t3[train_ix_]
    test_prop = (1/n_folds)/(1 - 1/n_folds)
    train_ix, val_ix = train_test_split(
        train_ix_, test_size=test_prop, 
        stratify=train_y, random_state=12345)
    # subset.
    train_dataset = Subset(risk_dataset_t3, train_ix)
    val_dataset = Subset(risk_dataset_t3, val_ix)
    test_dataset = Subset(risk_dataset_t3, test_ix)
    train_y = ys_t3[train_ix]
    val_y = ys_t3[val_ix]
    test_y = ys_t3[test_ix]
    # weighted sampler.
    f0, f1 = np.bincount(train_y)
    train_w = np.zeros_like(train_y, dtype='float')
    train_w[train_y==0] = 1/f0
    train_w[train_y==1] = 1/f1
    weighted_sampler = WeightedRandomSampler(
        train_w, len(train_y)//batch_size*batch_size, 
        replacement=True)
    # data loaders.
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, 
        num_workers=cpu_threads, sampler=weighted_sampler,
        collate_fn=mammo_collate)
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    # Reset model before training.
    model = init_model(multi_gpu=False)
    # train & test.
    best_name_ = 'best_model_{}.pt'.format(fold)
    print('='*10, 'Fold', fold, '='*10)
    _, start_auc = val_loss(model, test_loader, return_auc=True)
    start_auc_m = test_max_auc(model, test_loader)
    print('Test AUC at start={:.3f}, max-score-based AUC={:.3f}'.format(
        start_auc, start_auc_m))
    train(model, train_loader, val_loader, best_name_, 
          epochs=epochs, lr=1e-5, check_iters=10)
    print('Predicting on the test set...', end='')
    subject_list, exam_list, \
    pred_list, label_list, machine_list, \
    age_list, race_list, bmi_list, \
    birads_list, libra_list = do_test(model, test_loader)
    subject_pool.extend(subject_list)
    exam_pool.extend(exam_list)
    pred_pool.extend(pred_list)
    label_pool.extend(label_list)
    machine_pool.extend(machine_list)
    age_pool.extend(age_list)
    race_pool.extend(race_list)
    bmi_pool.extend(bmi_list)
    birads_pool.extend(birads_list)
    libra_pool.extend(libra_list)
    # test AUC. 

Assemble the results from all batches

In [ ]:
all_subj_t3 = np.concatenate(subject_pool)
all_exam_t3 = np.concatenate(exam_pool)
all_preds_t3 = torch.cat(pred_pool)
all_labels_t3 = torch.cat(label_pool)
all_preds_t3 = all_preds_t3.cpu().numpy()
all_labels_t3 = all_labels_t3.numpy()
all_probs_max_t3 = all_preds_t3.max(1)
all_machines_t3 = np.concatenate(machine_pool)
all_ages_t3 = np.concatenate(age_pool)
all_races_t3 = np.concatenate(race_pool)
all_bmis_t3 = np.concatenate(bmi_pool)
all_birads_t3 = np.concatenate(birads_pool)
all_libras_t3 = np.concatenate(libra_pool)

In [ ]:
from sklearn.metrics import roc_auc_score
print('N={}'.format(len(all_labels_t3)))
print('4view max AUC={:.3f}'.format(roc_auc_score(all_labels_t3, all_probs_max_t3)))

In [ ]:
all_subj_t3 = [ '{:06d}'.format(s) for s in all_subj_t3]
all_subj_t3[:3]

In [ ]:
import pandas as pd
df_t3 = pd.DataFrame.from_dict(
    {'subject': all_subj_t3, 'exam': all_exam_t3, 'is_ge': all_machines_t3, 
     'age': all_ages_t3, 'race': all_races_t3, 'bmi': all_bmis_t3, 
     'birads': all_birads_t3,})
df_t3.head()

In [ ]:
d_ = np.concatenate([all_preds_t3, all_labels_t3[:, np.newaxis]], axis=1)
d_[:3]

In [ ]:
df_t3 = pd.concat([df_t3, pd.DataFrame(d_)], axis=1)
df_t3.head()

In [ ]:
df_t3 = df_t3.rename(
    columns={0: 'ips-cc', 1: 'ips-mlo', 2: 'contra-cc', 
             3: 'contra-mlo', 4: 'is_case'})
df_t3 = df_t3.astype({'is_case': 'int8'})
df_t3.head()

In [ ]:
df_t3.to_csv('time_set/finetuned_pred_score_4view_T3.csv', index=False)

Also calculate the max score based AUC using the model without finetuning

In [ ]:
cpu_threads = 4
batch_size = 4

all_loader_t3 = DataLoader(
    risk_dataset_t3, batch_size=batch_size*2, 
    num_workers=cpu_threads, drop_last=False,
    collate_fn=mammo_collate)

In [ ]:
model_0 = init_model(multi_gpu=False)
all_auc_m = test_max_auc(model_0, all_loader_t3)
print('MIT unchanged max-score-based auc={:.3f}'.format(all_auc_m))

**Risk prediction 1+ years**

In [ ]:
risk_dataset_t1p = torch.load('time_set/risk_pred_GEHolo_4view_matchedSepCase_T1+.pt')
len(risk_dataset_t1p)

In [ ]:
# ys_t1p = np.array([ sample['label'].item() for sample in risk_dataset_t1p])

In [ ]:
# np.save('tmp/ys_t1p.npy', ys_t1p)

In [ ]:
ys_t1p = np.load('tmp/ys_t1p.npy')

In [ ]:
np.unique(ys_t1p, return_counts=True)

In [ ]:
from torch.optim import Adam

criterion = nn.NLLLoss(reduction='mean').to(device)
cpu_threads = 3
batch_size = 4
n_folds = 5
epochs = 20
subject_pool, exam_pool = [], []
pred_pool, label_pool, machine_pool = [], [], []
age_pool, race_pool, bmi_pool = [], [], []
birads_pool, libra_pool = [], []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1234)
fold = 0
for train_ix_, test_ix in skf.split(np.ones((len(ys_t1p), 1)), ys_t1p):
    fold += 1
    # train-val-test idx.
    train_y = ys_t1p[train_ix_]
    test_prop = (1/n_folds)/(1 - 1/n_folds)
    train_ix, val_ix = train_test_split(
        train_ix_, test_size=test_prop, 
        stratify=train_y, random_state=12345)
    # subset.
    train_dataset = Subset(risk_dataset_t1p, train_ix)
    val_dataset = Subset(risk_dataset_t1p, val_ix)
    test_dataset = Subset(risk_dataset_t1p, test_ix)
    train_y = ys_t1p[train_ix]
    val_y = ys_t1p[val_ix]
    test_y = ys_t1p[test_ix]
    # weighted sampler.
    f0, f1 = np.bincount(train_y)
    train_w = np.zeros_like(train_y, dtype='float')
    train_w[train_y==0] = 1/f0
    train_w[train_y==1] = 1/f1
    weighted_sampler = WeightedRandomSampler(
        train_w, len(train_y)//batch_size*batch_size, 
        replacement=True)
    # data loaders.
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, 
        num_workers=cpu_threads, sampler=weighted_sampler,
        collate_fn=mammo_collate)
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    # Reset model before training.
    model = init_model(multi_gpu=False)
    # train & test.
    best_name_ = 'tmp/best_model_{}.pt'.format(fold)
    print('='*10, 'Fold', fold, '='*10)
    _, start_auc = val_loss(model, test_loader, return_auc=True)
    start_auc_m = test_max_auc(model, test_loader)
    print('Test AUC at start={:.3f}, max-score-based AUC={:.3f}'.format(
        start_auc, start_auc_m))
    train(model, train_loader, val_loader, best_name_, 
          epochs=epochs, lr=1e-5, check_iters=10)
    print('Predicting on the test set...', end='')
    subject_list, exam_list, \
    pred_list, label_list, machine_list, \
    age_list, race_list, bmi_list, \
    birads_list, libra_list = do_test(model, test_loader)
    subject_pool.extend(subject_list)
    exam_pool.extend(exam_list)
    pred_pool.extend(pred_list)
    label_pool.extend(label_list)
    machine_pool.extend(machine_list)
    age_pool.extend(age_list)
    race_pool.extend(race_list)
    bmi_pool.extend(bmi_list)
    birads_pool.extend(birads_list)
    libra_pool.extend(libra_list)
    # test AUC.
    _, fold_auc = val_loss(model, test_loader, return_auc=True)
    fold_auc_m = test_max_auc(model, test_loader)
    print('Done')
    print('Test AUC after training={:.3f}, max-score-based AUC={:.3f}'.format(
        fold_auc, fold_auc_m))
    if fold < n_folds:
        print("\n")

In [ ]:
all_subj_t1p = np.concatenate(subject_pool)
all_exam_t1p = np.concatenate(exam_pool)
all_preds_t1p = torch.cat(pred_pool)
all_labels_t1p = torch.cat(label_pool)
all_preds_t1p = all_preds_t1p.cpu().numpy()
all_labels_t1p = all_labels_t1p.numpy()
all_probs_max_t1p = all_preds_t1p.max(1)
all_machines_t1p = np.concatenate(machine_pool)
all_ages_t1p = np.concatenate(age_pool)
all_races_t1p = np.concatenate(race_pool)
all_bmis_t1p = np.concatenate(bmi_pool)
all_birads_t1p = np.concatenate(birads_pool)
all_libras_t1p = np.concatenate(libra_pool)

In [ ]:
from sklearn.metrics import roc_auc_score
print('N={}'.format(len(all_labels_t1p)))
print('4view max AUC={:.3f}'.format(roc_auc_score(all_labels_t1p, all_probs_max_t1p)))

In [ ]:
all_subj_t1p = [ '{:06d}'.format(s) for s in all_subj_t1p]
all_subj_t1p[:3]

In [ ]:
import pandas as pd

df_t1p = pd.DataFrame.from_dict(
    {'subject': all_subj_t1p, 'exam': all_exam_t1p, 'is_ge': all_machines_t1p, 
     'age': all_ages_t1p, 'race': all_races_t1p, 'bmi': all_bmis_t1p, 
     'birads': all_birads_t1p,})
d_ = np.concatenate([all_preds_t1p, all_labels_t1p[:, np.newaxis]], axis=1)
df_t1p = pd.concat([df_t1p, pd.DataFrame(d_)], axis=1)
df_t1p = df_t1p.rename(
    columns={0: 'ips-cc', 1: 'ips-mlo', 2: 'contra-cc', 
             3: 'contra-mlo', 4: 'is_case'})
df_t1p = df_t1p.astype({'is_case': 'int8'})
df_t1p.head()

In [ ]:
df_t1p.to_csv('time_set/finetuned_pred_score_4view_T1p_2.csv', index=False)

In [ ]:
cpu_threads = 4
batch_size = 4

all_loader_t1p = DataLoader(
    risk_dataset_t1p, batch_size=batch_size*2, 
    num_workers=cpu_threads, drop_last=False,
    collate_fn=mammo_collate)

In [ ]:
model_0 = init_model(multi_gpu=False)
all_auc_m = test_max_auc(model_0, all_loader_t1p)
print('MIT unchanged max-score-based auc={:.3f}'.format(all_auc_m))

**Risk prediction <1 year**

In [ ]:
risk_dataset_t1 = torch.load('time_set/risk_pred_GEHolo_4view_matchedSepCase_T1.pt')
len(risk_dataset_t1)

In [ ]:
# ys_t1 = np.array([ sample['label'].item() for sample in risk_dataset_t1])

In [ ]:
# np.save('tmp/ys_t1.npy', ys_t1)

In [ ]:
ys_t1 = np.load('tmp/ys_t1.npy')

In [ ]:
np.unique(ys_t1, return_counts=True)

In [ ]:
from torch.optim import Adam

criterion = nn.NLLLoss(reduction='mean').to(device)
cpu_threads = 3
batch_size = 4
n_folds = 5
epochs = 20
subject_pool, exam_pool = [], []
pred_pool, label_pool, machine_pool = [], [], []
age_pool, race_pool, bmi_pool = [], [], []
birads_pool, libra_pool = [], []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=12345)
fold = 0
for train_ix_, test_ix in skf.split(np.ones((len(ys_t1), 1)), ys_t1):
    fold += 1
    # train-val-test idx.
    train_y = ys_t1[train_ix_]
    test_prop = (1/n_folds)/(1 - 1/n_folds)
    train_ix, val_ix = train_test_split(
        train_ix_, test_size=test_prop, 
        stratify=train_y, random_state=12345)
    # subset.
    train_dataset = Subset(risk_dataset_t1, train_ix)
    val_dataset = Subset(risk_dataset_t1, val_ix)
    test_dataset = Subset(risk_dataset_t1, test_ix)
    train_y = ys_t1[train_ix]
    val_y = ys_t1[val_ix]
    test_y = ys_t1[test_ix]
    # weighted sampler.
    f0, f1 = np.bincount(train_y)
    train_w = np.zeros_like(train_y, dtype='float')
    train_w[train_y==0] = 1/f0
    train_w[train_y==1] = 1/f1
    weighted_sampler = WeightedRandomSampler(
        train_w, len(train_y)//batch_size*batch_size, 
        replacement=True)
    # data loaders.
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, 
        num_workers=cpu_threads, sampler=weighted_sampler,
        collate_fn=mammo_collate)
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size*2, 
        num_workers=cpu_threads, drop_last=False,
        collate_fn=mammo_collate)
    # Reset model before training.
    model = init_model(multi_gpu=False)
    # train & test.
    best_name_ = 'tmp/best_model_{}.pt'.format(fold)
    print('='*10, 'Fold', fold, '='*10)
    _, start_auc = val_loss(model, test_loader, return_auc=True)
    start_auc_m = test_max_auc(model, test_loader)
    print('Test AUC at start={:.3f}, max-score-based AUC={:.3f}'.format(
        start_auc, start_auc_m))
    train(model, train_loader, val_loader, best_name_, 
          epochs=epochs, lr=1e-5, check_iters=10)
    print('Predicting on the test set...', end='')
    subject_list, exam_list, \
    pred_list, label_list, machine_list, \
    age_list, race_list, bmi_list, \
    birads_list, libra_list = do_test(model, test_loader)
    subject_pool.extend(subject_list)
    exam_pool.extend(exam_list)
    pred_pool.extend(pred_list)
    label_pool.extend(label_list)
    machine_pool.extend(machine_list)
    age_pool.extend(age_list)
    race_pool.extend(race_list)
    bmi_pool.extend(bmi_list)
    birads_pool.extend(birads_list)
    libra_pool.extend(libra_list)
    # test AUC.
    _, fold_auc = val_loss(model, test_loader, return_auc=True)
    fold_auc_m = test_max_auc(model, test_loader)
    print('Done')
    print('Test AUC after training={:.3f}, max-score-based AUC={:.3f}'.format(
        fold_auc, fold_auc_m))
    if fold < n_folds:
        print("\n")

In [ ]:
all_subj_t1 = np.concatenate(subject_pool)
all_exam_t1 = np.concatenate(exam_pool)
all_preds_t1 = torch.cat(pred_pool)
all_labels_t1 = torch.cat(label_pool)
all_preds_t1 = all_preds_t1.cpu().numpy()
all_labels_t1 = all_labels_t1.numpy()
all_probs_max_t1 = all_preds_t1.max(1)
all_machines_t1 = np.concatenate(machine_pool)
all_ages_t1 = np.concatenate(age_pool)
all_races_t1 = np.concatenate(race_pool)
all_bmis_t1 = np.concatenate(bmi_pool)
all_birads_t1 = np.concatenate(birads_pool)
all_libras_t1 = np.concatenate(libra_pool)

In [ ]:
from sklearn.metrics import roc_auc_score
print('N={}'.format(len(all_labels_t1)))
print('4view max AUC={:.3f}'.format(roc_auc_score(all_labels_t1, all_probs_max_t1)))

In [ ]:
all_subj_t1 = [ '{:06d}'.format(s) for s in all_subj_t1]
all_subj_t1[:3]

In [ ]:
import pandas as pd

df_t1 = pd.DataFrame.from_dict(
    {'subject': all_subj_t1, 'exam': all_exam_t1, 'is_ge': all_machines_t1, 
     'age': all_ages_t1, 'race': all_races_t1, 'bmi': all_bmis_t1, 
     'birads': all_birads_t1,})
d_ = np.concatenate([all_preds_t1, all_labels_t1[:, np.newaxis]], axis=1)
df_t1 = pd.concat([df_t1, pd.DataFrame(d_)], axis=1)
df_t1 = df_t1.rename(
    columns={0: 'ips-cc', 1: 'ips-mlo', 2: 'contra-cc', 
             3: 'contra-mlo', 4: 'is_case'})
df_t1 = df_t1.astype({'is_case': 'int8'})
df_t1.head()

In [ ]:
df_t1.to_csv('time_set/finetuned_pred_score_4view_T1.csv', index=False)

In [ ]:
cpu_threads = 4
batch_size = 4

all_loader_t1 = DataLoader(
    risk_dataset_t1, batch_size=batch_size*2, 
    num_workers=cpu_threads, drop_last=False,
    collate_fn=mammo_collate)

In [ ]:
model_0 = init_model(multi_gpu=False)
all_auc_m = test_max_auc(model_0, all_loader_t1)
print('MIT unchanged max-score-based auc={:.3f}'.format(all_auc_m))